In [45]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Add,
    Input,
    Conv2D,
    MaxPool2D,
    LeakyReLU,
    Concatenate,
    UpSampling2D,
    ZeroPadding2D,
    BatchNormalization)
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2

### Backbone of YoloV3

In [30]:
class BatchNormalization(BatchNormalization):
    
    def call(self, x, training=False):
        if not training:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)
        return super().call(x, training)

def DarknetConv(x, filters, size, downsample=False, activate=True, bn=True):

    if downsample:
        x = ZeroPadding2D(((1, 0), (1, 0)))(x)
        padding = 'valid'
        strides = 2
    else:
        padding = 'same'
        strides = 1
    
    x = Conv2D(filters=filters, kernel_size=size,
               strides=strides, padding=padding, use_bias=not bn,
               kernel_regularizer=l2(0.0005),
               kernel_initializer=tf.random_normal_initializer(stddev=0.01),
               bias_initializer=tf.constant_initializer(0.))(x)
    
    if bn:
        x = BatchNormalization()(x)
    if activate:
        x = LeakyReLU(alpha=0.1)(x)
    
    return x

def DarknetResidual(x, filters):
    
    short_cut = x
    x = DarknetConv(x, filters=filters//2, size=1)
    x = DarknetConv(x, filters=filters, size=3)
    x = Add()([short_cut, x])
    
    return x


def DarknetBlock(x, filters, blocks):
    x = DarknetConv(x, filters=filters, size=3, downsample=True)
    for _ in range(blocks):
        x = DarknetResidual(x, filters)
    
    return x

def Darknet(name=None):

    x = inputs = Input([None, None, 3])
    x = DarknetConv(x, filters=32, size=3)
    x = DarknetBlock(x, 64, 2)
    x = DarknetBlock(x, 128, 2)
    x = x_36 = DarknetBlock(x, 256, 8)
    x = x_61 = DarknetBlock(x, 512, 8)
    x = DarknetBlock(x, 1024, 4)
    
    return Model(inputs, (x_36, x_61, x), name=name)

### YoloV3 model

In [35]:
def YoloConv(filters, name=None):
    def yolo_conv(x_in):
        if isinstance(x_in, tuple):
            inputs = Input(x_in[0].shape[1:]), Input(x_in[1].shape[1:])
            x, x_skip = inputs

            # concat with skip connection
            x = DarknetConv(x, filters, 1)
            x = UpSampling2D(2)(x)
            x = Concatenate()([x, x_skip])
        else:
            x = inputs = Input(x_in.shape[1:])

        x = DarknetConv(x, filters, 1)
        x = DarknetConv(x, filters * 2, 3)
        x = DarknetConv(x, filters, 1)
        x = DarknetConv(x, filters * 2, 3)
        x = DarknetConv(x, filters, 1)
        return Model(inputs, x, name=name)(x_in)
    return yolo_conv

In [43]:
def YoloV3(size=None, classes=80):
    x = inputs = Input([size, size, 3], name='input')
    
    x_36, x_61, x = Darknet(name='yolo_darknet')(x)
    
    x = YoloConv(512, name='yolo_conv_0')(x)
    l_output = DarknetConv(x, filters=3*(classes + 5), size=1, activate=False, bn=False)
    

    x = YoloConv(256, name='yolo_conv_1')((x, x_61))
    m_output = DarknetConv(x, filters=3*(classes + 5), size=1, activate=False, bn=False)

    x = YoloConv(128, name='yolo_conv_2')((x, x_36))
    s_output = DarknetConv(x, filters=3*(classes + 5), size=1, activate=False, bn=False)
    
    return [l_output, m_output, s_output]

In [47]:
m = YoloV3(416)

In [48]:
m

[<tf.Tensor 'conv2d_588/Identity:0' shape=(None, 13, 13, 255) dtype=float32>,
 <tf.Tensor 'conv2d_595/Identity:0' shape=(None, 26, 26, 255) dtype=float32>,
 <tf.Tensor 'conv2d_602/Identity:0' shape=(None, 52, 52, 255) dtype=float32>]